# nanoGPT x Friends Notebook

## Exploring the dataset

In [2]:
text = open('data/Friends_Transcript.txt', 'r').read()

In [3]:
print('The length of the transcript is', len(text), 'chars. \n\nAnd the first 1000 chars are as follows: \n')
print(text[:1000])

The length of the transcript is 4899189 chars. 

And the first 1000 chars are as follows: 

THE ONE WHERE MONICA GETS A NEW ROOMATE (THE PILOT-THE UNCUT VERSION)
Written by: Marta Kauffman & David Crane
[Scene: Central Perk, Chandler, Joey, Phoebe, and Monica are there.]
Monica: There's nothing to tell! He's just some guy I work with!
Joey: C'mon, you're going out with the guy! There's gotta be something wrong with him!
Chandler: All right Joey, be nice. So does he have a hump? A hump and a hairpiece?
Phoebe: Wait, does he eat chalk?
(They all stare, bemused.)
Phoebe: Just, 'cause, I don't want her to go through what I went through with Carl- oh!
Monica: Okay, everybody relax. This is not even a date. It's just two people going out to dinner and- not having sex.
Chandler: Sounds like a date to me.
[Time Lapse]
Chandler: Alright, so I'm back in high school, I'm standing in the middle of the cafeteria, and I realize I am totally naked.
All: Oh, yeah. Had that dream.
Chandler: Then I look

As we can see we are dealing with almost 5M chars in the dataset, now we will create a vocabulary from this dataset using its unique characters.

### Vocab and vocab size

In [4]:
vocab = sorted(list(set(text)))
vocab_size = len(vocab)
print(''.join(vocab))
print(vocab_size)


 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[]^_`abcdefghijklmnopqrstuvwxyz{|}
94


Now we will create a character encoder. When trying to represent text with numbers (integers) you have a couple of choices to make, there is a trade-off between shorter representations and bigger vocabularies. 

We will keep it simple and use a character encoding, i.e, we will give every char from out vocab a number and then we will map this dictionary to encode our text. 

### Encoder

In [5]:
stoi = {ch:i for i,ch in enumerate(vocab)}
itos = {i:ch for i,ch in enumerate(vocab)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("house"))
print(decode(encode("house")))

[72, 79, 85, 83, 69]
house


As a result, we have a number for each char in the word we want to encode. 

Imagine we want need representations, let's suppose we are encoding the word _house_, with our encoder we would need an array with length 5, but if the encoder had an specific number for _ho_ and another one for _use_ we would simply need 2 numbers to represent the word. But in order to have those words in the vocab we would need bigger a vocab size, to match every possible combination of chars. This is the tradeoff we were talking about.

### Tensors (yeah!)

In [6]:
import torch

data = torch.tensor(encode(text), dtype = torch.long)
data[:1000]

tensor([53, 41, 38,  1, 48, 47, 38,  1, 56, 41, 38, 51, 38,  1, 46, 48, 47, 42,
        36, 34,  1, 40, 38, 53, 52,  1, 34,  1, 47, 38, 56,  1, 51, 48, 48, 46,
        34, 53, 38,  1,  9, 53, 41, 38,  1, 49, 42, 45, 48, 53, 14, 53, 41, 38,
         1, 54, 47, 36, 54, 53,  1, 55, 38, 51, 52, 42, 48, 47, 10,  0, 56, 82,
        73, 84, 84, 69, 78,  1, 66, 89, 27,  1, 46, 65, 82, 84, 65,  1, 44, 65,
        85, 70, 70, 77, 65, 78,  1,  7,  1, 37, 65, 86, 73, 68,  1, 36, 82, 65,
        78, 69,  0, 60, 52, 67, 69, 78, 69, 27,  1, 36, 69, 78, 84, 82, 65, 76,
         1, 49, 69, 82, 75, 13,  1, 36, 72, 65, 78, 68, 76, 69, 82, 13,  1, 43,
        79, 69, 89, 13,  1, 49, 72, 79, 69, 66, 69, 13,  1, 65, 78, 68,  1, 46,
        79, 78, 73, 67, 65,  1, 65, 82, 69,  1, 84, 72, 69, 82, 69, 15, 61,  0,
        46, 79, 78, 73, 67, 65, 27,  1, 53, 72, 69, 82, 69,  8, 83,  1, 78, 79,
        84, 72, 73, 78, 71,  1, 84, 79,  1, 84, 69, 76, 76,  2,  1, 41, 69,  8,
        83,  1, 74, 85, 83, 84,  1, 83, 

We converted the whole text into numbers using our encoder and store the value in a tensor object using *PyTorch*. Now we can check again the first 1000 chars from the dataset, but now converted to integers.

### Train-Test Split

In [7]:
train_n = int(len(data)*0.95)
train_ds = data[:train_n]
test_ds = data[train_n:]

As we have almost 5M chars in the dataset, we will do a 95/5 train/test split, because 5% is more than enough to validate and we will need as much as we can in the train.

## Bigram model attempt

In [52]:
torch.manual_seed(777)
context_length = 8 #How much are we feeding the transformer at the same time, also known as block size. In this case 8 numbers
batch_size = 4 #Independent sequences processed in parallel

def get_batch(split):
    data = train_ds if split == 'train' else test_ds
    ix = torch.randint(len(data) - context_length, (batch_size,))
    x = torch.stack([data[i:i+context_length] for i in ix])
    y = torch.stack([data[i+1:i+context_length+1] for i in ix])
    return x, y

xb,yb = get_batch('train')
print('inputs:\n',xb)
print('targets:\n',yb)

for b in range(batch_size):
    for t in range(context_length):
        context = xb[b, :t+1] # Row b, all columns until t+1 element (not included)
        target = yb[b,t] #Row b, element t
        print(f"When input is {context.tolist()}, the target is: {target}")

inputs:
 tensor([[72, 79, 87,  1, 67, 79, 77, 69],
        [75, 78, 79, 87, 15,  1, 42,  1],
        [85,  1, 68, 79, 73, 78, 71,  1],
        [79,  1, 42,  1, 72, 65, 86, 69]])
targets:
 tensor([[79, 87,  1, 67, 79, 77, 69,  1],
        [78, 79, 87, 15,  1, 42,  1, 65],
        [ 1, 68, 79, 73, 78, 71,  1, 84],
        [ 1, 42,  1, 72, 65, 86, 69,  1]])
When input is [72], the target is: 79
When input is [72, 79], the target is: 87
When input is [72, 79, 87], the target is: 1
When input is [72, 79, 87, 1], the target is: 67
When input is [72, 79, 87, 1, 67], the target is: 79
When input is [72, 79, 87, 1, 67, 79], the target is: 77
When input is [72, 79, 87, 1, 67, 79, 77], the target is: 69
When input is [72, 79, 87, 1, 67, 79, 77, 69], the target is: 1
When input is [75], the target is: 78
When input is [75, 78], the target is: 79
When input is [75, 78, 79], the target is: 87
When input is [75, 78, 79, 87], the target is: 15
When input is [75, 78, 79, 87, 15], the target is: 1
When 

In [53]:
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets = None):
        logits = self.token_embedding_table(idx)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # We will focus on the last element because this is a Bigram Model, so we fix T = -1
            logits = logits[:, -1, :] # (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) 
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 94])
tensor(4.9358, grad_fn=<NllLossBackward0>)


Generating text using Bigram

In [54]:
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


g`3u`>(9]hrCJ7ET:}xUl%!RF]v$B+lJ)XCB9u
#^IBNDeY2/XMYDUMv:qsoy#t0a^O29EO>hUfH^_N0a*o0%IpO*!( d
15z#"3


Creating an Optimizer

In [55]:
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-4)

In [62]:
batch_size = 64

for steps in range(100000):
    xb, yb = get_batch('train')
    
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.495844841003418


In [63]:
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


Bun NI's aie d Tothey, lothong g llleve acat adnde t an!
Romera stou: aya  t, dld thit ot.y hears fi
